In [36]:
import pandas as pd
from gensim import corpora
from gensim.models.ldamodel import LdaModel

In [37]:
df = pd.read_csv("/content/stories.csv")

In [38]:
df.head()

,body,topic
0,,['39822b5f-e37e-43e8-b997-7142fe55c3ea']
1,,['0d817400-3f5d-41e0-929c-c31fdbe75d31']
2,,['83a09c6b-5f2f-421f-ae50-b38acca7e008']
3,,['6fbf954a-03f9-4782-a65f-783271c9c447']
4,hello and welcome to BBC News a woman who gave...,"['83a09c6b-5f2f-421f-ae50-b38acca7e008', '9ff5..."


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5181 entries, 0 to 5180
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    5181 non-null   object
 1   topic   5181 non-null   object
dtypes: object(2)
memory usage: 81.1+ KB


In [40]:
df = df[df != " "]

In [41]:
df.isnull().sum()

body     29
topic     0
dtype: int64

In [42]:
df = df.dropna()
df = df.reset_index(drop=True)

In [43]:
df['topic'] = df['topic'].apply(lambda x: [s.strip("'") for s in x.strip('[]').split('-')])
# Create a dictionary of words from the dataset
dictionary = corpora.Dictionary(df['body'].apply(lambda x: x.split()))

corpus = [dictionary.doc2bow(text.split()) for text in df['body']]

lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)

# Predict the topics for each document in the dataset
topic_predictions = []
for doc in corpus:
    topic_predictions.append(lda_model.get_document_topics(doc))

# Extract the most probable topics for each document
topic_ids = [[dictionary[topic[0]] for topic in sorted(topics, key=lambda x: x[1], reverse=True)[:3]] for topics in topic_predictions]

df['predicted_topics'] = topic_ids

In [44]:
df

,body,topic,predicted_topics
0,hello and welcome to BBC News a woman who gave...,"[83a09c6b, 5f2f, 421f, ae50, b38acca7e008', '9...","[Farmer,, Jeffrey]"
1,news now out of North Hollywood. A 14 yearold ...,"[9ff54ded, 904b, 4e0c, 85ce, a3617f5cb913]","[Maxwell, Epstein, Jeffrey]"
2,homelessness his city's greatest failure. That...,"[83a09c6b, 5f2f, 421f, ae50, b38acca7e008', '7...","[Gillen, New, Annie]"
3,Minneapolis police officer Kim Potter guilty o...,"[83a09c6b, 5f2f, 421f, ae50, b38acca7e008', '9...","[Maxwell, Jeffrey, Farmer,]"
4,Judy an update now to the wildfires that wiped...,"[9ff54ded, 904b, 4e0c, 85ce, a3617f5cb913', '9...","[Farmer,, Gillen, York]"
...,...,...,...
5147,News. More local help will soon be on the way....,"[9ff54ded, 904b, 4e0c, 85ce, a3617f5cb913', '9...","[Gillen, Maxwell, Annie]"
5148,"with March 1, we start what is called Meteorol...","[9a06646a, e1df, 4fca, 888e, 69658420556b]","[New, Farmer,, Gillen]"
5149,overseas. A massive Russian convoy is headed t...,"[9ff54ded, 904b, 4e0c, 85ce, a3617f5cb913]","[Gillen, York]"
5150,"And this morning, the National Hockey League s...","[9ff54ded, 904b, 4e0c, 85ce, a3617f5cb913', 'b...","[Gillen, Annie, Jeffrey]"
